In [1]:
import pandas as pd
import requests 
import json
from pandas.io.json import json_normalize
import os
import scipy.linalg as linalg
from pulp import *
import csv
import itertools
from tqdm import tqdm
import statsmodels.stats.moment_helpers as stats
from scipy import stats as scipy_stats
import scipy
import matplotlib.pyplot as plt
import math
import matplotlib.mlab as mlab
from datetime import datetime
import numpy as np
from random import randint
from collections import Counter
os.chdir('/home/valesco/Datasets/NFL_projs')
pd.set_option('display.max_columns', 106)
pd.set_option('display.max_rows', 100)

nfl_week = '2017_week_7'

In [31]:
url = 'http://www.fantasylabs.com/api/playermodel/1/10_18_2017/?modelId=1196314&projOnly=true' 

headers = { 'User-agent': 'Mozilla/5.0',
            'Cookie':'__cfduid=d2e9d0d941e292062b5792a57cd6183f11505095681; LD_U=http%3A%2F%2Fwww.fantasylabs.com%2F; __distillery=ddea8bf_c6fa751c-2767-47ec-9073-7092b00d479d-0862dfd0d-8888296cfa38-33df; LD_R=http%3A%2F%2Fwww.fantasylabs.com%2Fnfl%2Fcontests-dashboard%2F; .AspNet.Cookies=ND2gLWxhZFBHI7JZZ_4IVNiZoac0nuMsQIGn3DzhbqFX28_lFq85ZSd8gycI05VNPbkRdB2Dn0sHs2Lso3cHrZA2f7wT8Y4-uayJBp42j9dpAxLuhZIPIvSljlzs-j_3e_c_R3FELThFTHZ0VYdkhZhCgdrvvXhZuBIil-UI3dMpSZCeps89DMC4wESmhvL6yLNg5qZhJaVpH3CjA4pe1-zcLyKrGOUi5eck31W5suU0l6nJdbHScIUcilgDO6uvB_YkVB0Q3Nt47NPn6pJrgmRDxHjwE-h68w0mgPoVSqqMyJlDaFADQqGOvs6tiepofNB4XidtFOg2CIfTVutAvRM8APkQCB0JI1iM2KiQ7Q3cvvM7WGYsuYzHcvZVdBe0zYiqUtJn0__45fCu5SlYe8aNXVHi6plyvW3Tc0YL0uLnmEBzhfLANPrc5A1SkDW0; flid=V2y0f5FbyE2AwjnR1hOc8Q; __zlcmid=iSgjnvnLj8Z3xC; LD_T=8265517e-bdb6-44e0-fd38-0f5bcbdfb4a5; _ga=GA1.2.993633943.1505095683; _gid=GA1.2.136241772.1507953815; hasmembership=True'
           }


response = requests.get(url, headers = headers)
json_data = response.json()

week_all = json_normalize(json_data['PlayerModels'])

week_dk = week_all.loc[week_all['Properties.SourceId'] == 4]

cols = ['PlayerId',
 'Properties.Player_Name',
 'Salary',
 'TeamName',
 'Properties.Opposing_TeamFB',
 'Properties.AvgPts',
 'Properties.Floor',
 'Properties.FloorPct',
 'Properties.FloorPtPerDPct',
 'Properties.Ceiling',
 'Properties.CeilingPct',
 'Properties.Consistency',
 'Position',
 'PositionCount',
 'PositionId',
 'Positions',
 'Properties.AdjYPA',
 'Properties.EventTeamId',
 'Properties.ExpectedSacksPct',
 'Properties.ImpPts',
 'Properties.InterceptionPct',
 'Properties.IsProjected',
 'Properties.MktShrPct',
 'Properties.Month_FPOpp',
 'Properties.Month_FPSnap',
 'Properties.OffensiveSnapsMarketShare',
 'Properties.OppPlusMinusPct',
 'Properties.OpportunitiesRedZone',
 'Properties.OpportunitiesRedZone10',
 'Properties.OpportunitiesRedZone5',
 'Properties.PassINTPct',
 'Properties.PassSackPct',
 'Properties.PassTDPct',
 'Properties.PassingAttemptsRBPct',
 'Properties.PassingAttemptsTEPct',
 'Properties.PassingAttemptsWRPct',
 'Properties.PassingSuccessfulPct',
 'Properties.Plus_Minus',
 'Properties.Position',
 'Properties.PositionId',
 'Properties.Pro_Pct',
 'Properties.ProjPct',
 'Properties.ProjPlaysPct',
 'Properties.ProjPlusMinusPct',
 'Properties.PtPerDPct',
 'Properties.ReceivingTargetsMarketShare',
 'Properties.ReceivingTouchdownsMarketShare',
 'Properties.ReceivingTouchdownsRedZonePct',
 'Properties.ReceivingYardsMarketShare',
 'Properties.RedZoneSnaps',
 'Properties.RedZoneSnaps10',
 'Properties.RedZoneSnaps5',
 'Properties.RedZoneTouchdownPct',
 'Properties.RushingFantasyPct',
 'Properties.RushingSuccessfulAllowedPct',
 'Properties.RushingSuccessfulPct',
 'Properties.RushingSuccessfulPctRedZone',
 'Properties.RushingTouchdownsMarketShare',
 'Properties.RushingTouchdownsRedZonePct',
 'Properties.RushingYardsMarketShare',
 'Properties.SackPct',
 'Properties.Salary',
 'Properties.Salary_Movement',
 'Properties.Score',
 'Properties.Season_FPOpp',
 'Properties.Season_FPSnap',
 'Properties.Season_PPG',
 'Properties.Season_PPG_Percentile',
 'Properties.Season_Plus_Minus',
 'Properties.Season_Salary_Change',
 'Properties.Season_X0',
 'Properties.Season_X1',
 'Properties.Season_X2',
 'Properties.Site_Salary',
 'Properties.SourceId',
 'Properties.TakeawayPct',
 'Properties.Team',
 'Properties.TouchdownAllowedPct',
 'Properties.Upside',
 'Properties.Vegas',
 'Properties.YardsPerPassingAttempt',
 'Score']

week_dk = week_dk[cols]

new_cols = ['player_id',
 'name',
 'salary',
 'team',
 'opponent',
 'points_avg',
 'points_floor',
 'floor_pct',
 'floor_pt_per_dpct',
 'points_ceiling',
 'ceiling_pct',
 'consistency',
 'position',
 'position_count',
 'position_id',
 'positions',
 'adj_ypa',
 'event_team_id',
 'expected_sack_pct',
 'Properties.ImpPts',
 'Properties.InterceptionPct',
 'Properties.IsProjected',
 'Properties.MktShrPct',
 'Properties.Month_FPOpp',
 'Properties.Month_FPSnap',
 'Properties.OffensiveSnapsMarketShare',
 'Properties.OppPlusMinusPct',
 'Properties.OpportunitiesRedZone',
 'Properties.OpportunitiesRedZone10',
 'Properties.OpportunitiesRedZone5',
 'Properties.PassINTPct',
 'Properties.PassSackPct',
 'Properties.PassTDPct',
 'Properties.PassingAttemptsRBPct',
 'Properties.PassingAttemptsTEPct',
 'Properties.PassingAttemptsWRPct',
 'Properties.PassingSuccessfulPct',
 'Properties.Plus_Minus',
 'Properties.Position',
 'Properties.PositionId',
 'Properties.Pro_Pct',
 'Properties.ProjPct',
 'Properties.ProjPlaysPct',
 'Properties.ProjPlusMinusPct',
 'Properties.PtPerDPct',
 'Properties.ReceivingTargetsMarketShare',
 'Properties.ReceivingTouchdownsMarketShare',
 'Properties.ReceivingTouchdownsRedZonePct',
 'Properties.ReceivingYardsMarketShare',
 'Properties.RedZoneSnaps',
 'Properties.RedZoneSnaps10',
 'Properties.RedZoneSnaps5',
 'Properties.RedZoneTouchdownPct',
 'Properties.RushingFantasyPct',
 'Properties.RushingSuccessfulAllowedPct',
 'Properties.RushingSuccessfulPct',
 'Properties.RushingSuccessfulPctRedZone',
 'Properties.RushingTouchdownsMarketShare',
 'Properties.RushingTouchdownsRedZonePct',
 'Properties.RushingYardsMarketShare',
 'Properties.SackPct',
 'Properties.Salary',
 'Properties.Salary_Movement',
 'Properties.Score',
 'Properties.Season_FPOpp',
 'Properties.Season_FPSnap',
 'Properties.Season_PPG',
 'Properties.Season_PPG_Percentile',
 'Properties.Season_Plus_Minus',
 'Properties.Season_Salary_Change',
 'Properties.Season_X0',
 'Properties.Season_X1',
 'Properties.Season_X2',
 'Properties.Site_Salary',
 'Properties.SourceId',
 'Properties.TakeawayPct',
 'Properties.Team',
 'Properties.TouchdownAllowedPct',
 'Properties.Upside',
 'Properties.Vegas',
 'Properties.YardsPerPassingAttempt',
 'Score']

week_dk.columns = new_cols


def home_away(x):
    if '@' in x:
        return 'away'
    else:
        return 'home'

proj_df = week_dk
cov_df = pd.read_csv('nfl_covariance_matrix.csv', index_col = 0)

#number of games in spreadsheet plus one
#num_of_games = 14


proj_df['matchup'] = tuple(zip(proj_df['team'], proj_df['opponent']))
#proj_df['salary'] = proj_df['salary'].apply(lambda x: x.replace('$',''))
#proj_df['salary'] = proj_df['salary'].apply(lambda x: float(x.replace('K', '')))
#proj_df['salary'] = proj_df['salary'].apply(lambda x: x * 1000)
proj_df['rank'] = proj_df.groupby(['team', 'position'])['points_avg'].rank(ascending = False, method = 'max')

proj_df['rank'].fillna(value = 9999, inplace = True)
proj_df['salary'].fillna(value = 9999, inplace = True)
proj_df['points_avg'].fillna(value = -9.999, inplace = True)
proj_df['rank'] = proj_df['rank'].apply(lambda x: str(int(x)))
proj_df['home_away'] = proj_df['opponent'].apply(lambda x: home_away(x))
proj_df['rank'] = proj_df['home_away'] + '_' + proj_df['position'] + proj_df['rank']

players = list(proj_df['name'].values)
salary = list(proj_df['salary'].values)
points = list(proj_df['points_avg'].values)
position = list(proj_df['position'].values)

player_dict = dict((z[0], list(z[1:])) for z in zip(players, salary, points, position))


away_teams = proj_df['team'].loc[proj_df['home_away'] == 'away'].unique()

print(away_teams)

player_cols = []
count = 0

for team in away_teams:
    temp1 = proj_df.loc[proj_df['team'] == team]
    temp2 = proj_df.loc[proj_df['opponent'] == team]
    temp_df = pd.concat([temp1, temp2], axis = 0)
    
    cols = cov_df.columns

    player_ls = []
    for c in cols:
        player = pd.DataFrame(temp_df.loc[temp_df['rank'] == c])
        
        if len(player) < 1:
            name = 'Not Available_' + str(count)
            count += 1
        else:
            name = player.iloc[0]['name']
        player_ls.append(name)
        
    player_cols.extend(player_ls)
    

arr_cov = np.asarray(cov_df)
for i in range(1, 15):
    arr_new = np.asarray(cov_df)
    arr_cov = linalg.block_diag(arr_cov, arr_new)
    
comp_cov_df = pd.DataFrame(data = arr_cov, index = player_cols, columns = player_cols)

position_std = {'QB': 8.096014, 'WR': 9.813933, 'RB': 8.702397, 'TE': 8.096014, 'D': 6.09265}
position_mu = {'QB': 19.861207, 'WR': 12.67457, 'RB': 12.900904, 'TE': 12.50152, 'D': 7.447265}

# Set position lists
qbs = []
wrs = []
rbs = []
tes = []
dsts = []
flexs = [] 

for player, ls in player_dict.items():
    if ls[2] == 'QB':
        if ls[1] > 15:
            qbs.append(player)
    if ls[2] == 'WR':
        if ls[1] > 7:
            wrs.append(player)
    if ls[2] == 'RB':
        if ls[1] > 8:
            rbs.append(player)
    if ls[2] == 'TE':
        if ls[1] > 7:
            tes.append(player)
    if ls[2] == 'D':
        if ls[1] > 4:
            dsts.append(player)
    if ls[2] == 'WR' or ls[2] == 'TE' or ls[2] == 'WR':
        if ls[1] > 8:
            flexs.append(player)
            
def create_first_team(mydict, **kwargs):
    while True:
        qb = qbs[randint(0,len(qbs) - 1)]
        wr1 = wrs[randint(0,len(wrs) - 1)]
        wr2 = wrs[randint(0,len(wrs) - 1)]
        wr3 = wrs[randint(0,len(wrs) - 1)]
        rb1 = rbs[randint(0,len(rbs) - 1)]
        rb2 = rbs[randint(0,len(rbs) - 1)]
        te = tes[randint(0,len(tes) - 1)]
        d = dsts[randint(0,len(dsts) - 1)]
        flex = flexs[randint(0,len(flexs) - 1)]

        player_list =  [qb, rb1, rb2, wr1, wr2, wr3, te, flex, d]
            
        team_salary = 0

        for player in player_list:
            team_salary += player_dict[player][0]

        duplicates = Counter(player_list)

        if team_salary <= 50000 and team_salary >= 47000 and sum(duplicates.values()) == 9:
            for key, value in mydict.items():
                if key == 'QB':
                    z = 0
                elif key == 'RB1':
                    z = 1
                elif key == 'RB2':
                    z = 2
                elif key == 'WR1': 
                    z = 3
                elif key == 'WR2':
                    z = 4
                elif key == 'WR3':
                    z = 5
                elif key == 'TE':
                    z = 6
                elif key == 'FLEX':
                    z = 7
                elif key == 'DST' or key == 'D' or key == 'DEF':
                    z = 8
                else:
                    print('Waaat')

                player_list[z] = value
            break
                
    return player_list

#Simulated annealing
def update_temp(i, t):
    temp = t / log(i)
    return temp

def cost(ls):
    qb = ls[0]
    rb1 = ls[1]
    rb2 = ls[2]
    wr1 = ls[3]
    wr2 = ls[4]
    wr3 = ls[5]
    te = ls[6]
    flex = ls[7]
    dst = ls[8]
    flex_pos = player_dict[flex][2]
    
    qb_mu = player_dict[qb][1]
    wr1_mu = player_dict[wr1][1]
    wr2_mu = player_dict[wr2][1]
    wr3_mu = player_dict[wr3][1]
    rb1_mu = player_dict[rb1][1]
    rb2_mu = player_dict[rb2][1]
    te_mu = player_dict[te][1]
    dst_mu = player_dict[dst][1]
    flex_mu = player_dict[flex][1]
    
    qb_std = position_std['QB']
    wr_std = position_std['WR']
    rb_std = position_std['RB']
    te_std = position_std['TE']
    dst_std = position_std['D']
    
    if flex_pos == 'WR':
        flex_std = wr_std
    elif flex_pos == 'RB':
        flex_std = rb_std
    elif flex_pos == 'TE':
        flex_std = te_std
    else:
        print('Waat')
        
    #create covariance matrix
    corr_array = []
    
    
    for player1 in ls:
        temp_ls = []
        for player2 in ls:
            try:
                corr = comp_cov_df[player1][player2]
            except:
                corr = 0
            temp_ls.append(corr)
            
        corr_array.append(temp_ls)
        
    corr_matrix = np.array(corr_array)
    cov_matrix = stats.corr2cov(corr_matrix, [qb_std, wr_std, wr_std, wr_std, rb_std, rb_std, te_std,
                                             dst_std, flex_std])
    cov_sum = 0
    
    for i in range(9):
        for x in range(9):
            if i > x:
                cov_sum += cov_matrix[x,i]
    
    #Calculate team variance
    
    team_var = (qb_std ** 2 + wr_std ** 2 + wr_std ** 2 + wr_std ** 2 + rb_std ** 2 + rb_std ** 2 +
                te_std ** 2 + dst_std ** 2 + flex_std ** 2 + 2 * cov_sum)
    
    team_mu = qb_mu + wr1_mu + wr2_mu + wr3_mu + rb1_mu + rb2_mu + te_mu + dst_mu + flex_mu
    
    team_ppf = scipy_stats.norm(team_mu, np.sqrt(team_var)).ppf(.95)
    
    return team_ppf

def random_except(length, except_ls):
    while True:
        r = randint(0, length - 1 )
        if r not in except_ls:
            return r
            break

    
    
def create_team(ls, except_ls, player_except):
    counter = 0
    while True:
        temp_best = ls[:]
        r = random_except(len(temp_best), except_ls)

        if r == 0:
            new_player = qbs[randint(0,len(qbs) - 1)]
        elif r > 0 and r <= 2:
            new_player = rbs[randint(0, len(rbs) - 1)]
        elif r > 2 and r <= 5:
            new_player = wrs[randint(0, len(wrs) - 1)]
        elif r == 6:
            new_player = tes[randint(0, len(tes) - 1)]
        elif r == 7:
            new_player = flexs[randint(0, len(flexs) - 1)]
        elif r == 8:
            new_player = dsts[randint(0, len(dsts) - 1)]
        else:
            print('Waaat')

        temp_best[r] = new_player

        team_salary = 0

        for player in temp_best:
            team_salary += player_dict[player][0]

        duplicates = Counter(temp_best)

        if team_salary <= 50000 and team_salary >= 47000 and len(duplicates.values()) == 9:
                break
                
        counter += 1
        
        if counter > 1000:
            temp_best = create_first_team(player_except)
            break
            
    return temp_best

['ATL' 'WAS' 'KC' 'CIN' 'TB' 'NO' 'BAL' 'TEN' 'NYJ' 'JAX' 'ARI' 'CAR' 'DAL'
 'SEA' 'DEN']


In [32]:
num_iterations = 200
points_to_subtract = .5
percent_iter_to_reduce_points = .10
reduce_points_iter = round(num_iterations * percent_iter_to_reduce_points)

#create df to track points per iteration
track_points_df = pd.DataFrame()
track_points_df['player'] = players
track_points_df['points_start'] = points

for iteration in tqdm(range(num_iterations)):
    final_teams = []
    except_ls = []
    player_except = {}
    
    #calc points to reduce starting reduction points
    #if iteration % reduce_points_iter == 0 and iteration != 0:
        #points_to_subtract = points_to_subtract * percent_iter_to_reduce_points + .05
    

    for z in range(30):
        player_list = create_first_team(player_except)

        max_iters = 2000
        best_team = player_list[:]
        current_best = player_list[:]
        temp = 100
        temp_change = .995
        ppf_list = []
        temp_ls = []
        count = 0

        for i in range(max_iters):
            new_team = create_team(current_best, except_ls, player_except)   

            cost_new = cost(new_team)
            cost_curr = cost(current_best)

            if cost_new > cost_curr:
                current_best = new_team[:]
                ppf_list.append([i, cost_new])
            else:
                temp = temp * temp_change
                if temp < 0:
                    temp = 100 - count
                    if count < 100:
                        count += 7
                r = randint(1,120)

                if r < temp:
                    temp_ls.append([r, temp])
                    current_best = new_team[:]
                    ppf_list.append([i, cost_new])

        pff_current = cost(current_best)
        current_best.append(pff_current)
        final_teams.append(current_best)


    final_df = pd.DataFrame(columns = ['QB', 'RB', 'RB', 'WR', 'WR', 'WR', 'TE', 'FLEX', 'DST', 'PPF'])


    for a, team in enumerate(final_teams):
        final_df.loc[a] = final_teams[a]
        
    final_df = pd.DataFrame(columns = ['QB', 'RB1', 'RB2', 'WR1', 'WR2', 'WR3', 'TE', 'FLEX', 'DST', 'PPF'])


    for i, team in enumerate(final_teams):
        final_df.loc[i] = final_teams[i]
        
    pos_list = ['QB', 'RB1', 'RB2', 'WR1', 'WR2', 'WR3', 'TE', 'FLEX', 'DST']
    grouped_df = final_df.groupby(pos_list[0]).mean()

    for position in pos_list[1:]:
        grouped_df = grouped_df.append(final_df.groupby(position).mean())

    grouped_df.reset_index(inplace = True)
    final_grouped = grouped_df.groupby('index').mean()
    final_grouped.reset_index(inplace = True)
    final_grouped.columns = ['player', 'ppf']

    final_grouped.sort_values('ppf', ascending = False, inplace = True)
    final_grouped['rank'] = final_grouped['ppf'].rank(ascending = False)

    final_grouped['points_to_minus'] = np.linspace(1, 0, len(final_grouped)) * points_to_subtract
    total_points_added = final_grouped['points_to_minus'].sum()
    
    players_in_grouped = final_grouped['player'].values
    
    final_df.sort_values('PPF', ascending = False, inplace = True)
    
    print(final_df.iloc[0], points_to_subtract)
    
    #create temp_dict to map to track df
    temp_player_dict = {}
    for key in player_dict.keys():
        if key in players_in_grouped:
            player_points_to_subtract = final_grouped['points_to_minus'].loc[final_grouped['player'] == key].values[0]
            player_dict[key][1] = player_dict[key][1] - player_points_to_subtract
        else:
            player_dict[key][1] += total_points_added / (len(list(player_dict.keys())) - len(players_in_grouped))
            
        temp_player_dict[key] = player_dict[key][1]
    
    track_points_col_name = 'iteration{}'.format(iteration)
    track_points_df[track_points_col_name] = track_points_df['player'].map(temp_player_dict)
    

  0%|          | 1/200 [03:10<10:33:03, 190.87s/it]

QB        Marcus Mariota
RB1         Le'Veon Bell
RB2          Mark Ingram
WR1     Rishard Matthews
WR2          Eric Decker
WR3        Ricardo Louis
TE           Evan Engram
FLEX    Demaryius Thomas
DST        Miami Defense
PPF              179.861
Name: 9, dtype: object 0.5


  1%|          | 2/200 [06:22<10:30:53, 191.18s/it]

QB        Marcus Mariota
RB1         LeSean McCoy
RB2         Le'Veon Bell
WR1        Bennie Fowler
WR2     Rishard Matthews
WR3     Demaryius Thomas
TE           Evan Engram
FLEX         Eric Decker
DST        Miami Defense
PPF              176.841
Name: 2, dtype: object 0.5


  2%|▏         | 3/200 [09:33<10:27:24, 191.09s/it]

...

QB          Philip Rivers
RB1       Ezekiel Elliott
RB2     Leonard Fournette
WR1       Tyrell Williams
WR2          Torrey Smith
WR3          Keenan Allen
TE             Jack Doyle
FLEX     Laquon Treadwell
DST        Dallas Defense
PPF               11.1634
Name: 15, dtype: object 0.5


100%|█████████▉| 199/200 [10:00:04<03:00, 180.86s/it]

QB             Carson Palmer
RB1              Carlos Hyde
RB2            Derrick Henry
WR1              Jaron Brown
WR2         Larry Fitzgerald
WR3         Sterling Shepard
TE                Ed Dickson
FLEX              John Brown
DST     Jacksonville Defense
PPF                  11.7075
Name: 17, dtype: object 0.5


100%|██████████| 200/200 [10:03:04<00:00, 180.76s/it]

QB               Russell Wilson
RB1           Wendell Smallwood
RB2             Latavius Murray
WR1                Doug Baldwin
WR2                  Mike Evans
WR3                Cole Beasley
TE      Austin Seferian-Jenkins
FLEX              Tyler Lockett
DST            Carolina Defense
PPF                     8.77609
Name: 2, dtype: object 0.5


In [33]:
track_points_df

,player,points_start,iteration0,iteration1,iteration2,iteration3,iteration4,iteration5,iteration6,iteration7,iteration8,iteration9,iteration10,iteration11,iteration12,iteration13,iteration14,iteration15,iteration16,iteration17,iteration18,iteration19,iteration20,iteration21,iteration22,iteration23,iteration24,iteration25,iteration26,iteration27,iteration28,iteration29,iteration30,iteration31,iteration32,iteration33,iteration34,iteration35,iteration36,iteration37,iteration38,iteration39,iteration40,iteration41,iteration42,iteration43,iteration44,iteration45,iteration46,iteration47,iteration48,iteration49,iteration50,...,iteration147,iteration148,iteration149,iteration150,iteration151,iteration152,iteration153,iteration154,iteration155,iteration156,iteration157,iteration158,iteration159,iteration160,iteration161,iteration162,iteration163,iteration164,iteration165,iteration166,iteration167,iteration168,iteration169,iteration170,iteration171,iteration172,iteration173,iteration174,iteration175,iteration176,iteration177,iteration178,iteration179,iteration180,iteration181,iteration182,iteration183,iteration184,iteration185,iteration186,iteration187,iteration188,iteration189,iteration190,iteration191,iteration192,iteration193,iteration194,iteration195,iteration196,iteration197,iteration198,iteration199
0,Atlanta Defense,4.0,4.072545,4.147995,4.235612,4.343285,4.444013,4.570315,4.687067,4.821375,4.968353,5.102662,5.254051,5.382323,5.538222,5.689611,5.836589,5.979252,6.117693,6.273591,6.409955,6.570466,6.724098,6.882290,7.031461,7.189653,7.321928,7.484785,7.627448,7.776620,7.921429,8.077327,8.211636,8.354299,8.510198,8.638470,8.772779,8.905054,9.027476,9.170139,9.285038,9.423479,9.553742,9.709641,9.833994,9.966269,10.072180,10.202443,10.353832,10.484095,10.593548,10.725823,10.837073,...,23.539614,23.702472,23.847280,24.000912,24.145720,24.315778,24.467167,24.632397,24.788296,24.933105,25.093616,25.223879,25.360243,25.535243,25.684414,25.827077,25.994707,26.148339,26.301970,26.448948,26.614178,26.779408,26.919948,27.073580,27.220558,27.383415,27.546272,27.686812,27.864327,28.029557,28.176535,28.327924,28.507984,28.661615,28.831673,28.989865,29.145764,29.318279,29.452587,29.617817,29.773716,29.925105,30.087962,30.241593,30.416593,30.572492,30.752552,30.917781,31.090296,31.241685,31.429564,31.559827,31.715726
1,Tevin Coleman,12.3,12.167188,12.000521,11.993764,11.825159,11.660525,11.786827,11.759355,11.619355,11.525015,11.659324,11.390805,11.380496,11.221405,11.026961,11.173939,10.919131,11.057572,11.213471,10.931292,11.091804,10.935840,11.094033,11.243204,11.130591,11.262866,11.010654,11.153317,11.302488,10.997726,11.153625,11.048625,10.942856,11.098755,11.227027,10.937027,11.069302,10.957600,11.100263,10.816930,10.955371,11.085634,10.794725,10.919077,11.051352,10.780764,10.911027,10.494361,10.624624,10.734076,10.866352,10.977602,...,8.677789,8.217612,8.362421,8.114714,8.259523,8.052627,8.204016,8.369245,8.123791,8.109505,7.872898,8.003161,7.537815,7.712815,7.861986,7.626409,7.794039,7.564681,7.555507,7.702485,7.298976,7.154239,7.294780,6.923220,7.070198,6.840110,6.397632,6.538172,6.715687,6.880917,6.753559,6.744299,6.640133,6.793764,6.543764,6.426647,6.331192,6.108970,6.243279,5.743279,5.899178,6.050566,6.213424,6.167552,6.342552,6.033461,6.213521,5.875801,5.645032,5.404291,5.592170,5.722433,5.286070
2,Taylor Gabriel,9.0,9.072545,9.147995,9.235612,9.343285,9.245724,9.372026,9.488778,9.453778,9.378306,9.512614,9.211688,9.026121,9.182020,8.955168,8.945734,8.614004,8.584592,8.270956,8.023431,8.183942,8.337573,8.170907,8.110159,8.268351,7.889563,7.823192,7.352038,7.300636,7.445445,7.022717,7.157026,6.988757,6.588757,6.717029,6.217029,6.349304,6.471727,6.614390,6.119945,5.757200,5.887463,6.043362,5.548625,5.680901,5.439724,5.569987,5.074617,5.204880,5.314333,5.446608,5.557858,...,-3.432262,-3.269405,-3.683690,-3.530059,-4.015773,-3.845715,-4.114234,-4.320374,-4.334011,-4.586391,-4.787284,-4.879121,-5.339517,-5.164517,-5.015346,-5.486500,-5.318

In [20]:
points_to_subtract

0.0